# Metadata Ontology for WWTFs

Outline
- Problem Definition
- Graph-based Metadata
- Application Examples

## Problem Definition

- Rising need for data-driven decision making for WWTFs
- More data available for WWTF through sensors, actuators
  - $\rightarrow$ need to manage this data!
- Identifiers, properties, data source descriptions are distributed across many sources
  - PFD, P&ID, SCADA, others
- Solution: simplify data management by unifying separate descriptions of the same WWTF

## Solution Overview

<img style="width: 50%" src="fig/metadata-graph.png" />

- represent equipment, unit processes, pipes, etc as *nodes* in a graph
- rich relationships between these entities
- use common labels to join different representations together

## Modeling Setup

- use BuildingMOTIF library from NREL to create our graph of a water treatment system

```python
from buildingmotif import BuildingMOTIF
from buildingmotif.model_builder import TemplateBuilderContext
from buildingmotif.dataclasses import Library, Model
from buildingmotif.namespaces import bind_prefixes
from rdflib import Namespace
from utils import write_to_file

# setup our buildingmotif instance
bm = BuildingMOTIF("sqlite://", shacl_engine="topquadrant")
```

## Modeling Setup (2)

- create the Model to hold our graph
- setup libraries of templates

```python
# create the model w/ a namespace
BLDG = Namespace("urn:nrel_example/")
bldg = Model.create(BLDG)
bind_prefixes(bldg.graph)
bldg.graph.bind("bldg", BLDG)

nrel = Library.load(directory="BuildingMOTIF/libraries/ashrae/223p/nrel-templates")
water = Library.load(directory="water-templates")
s223 = Library.load(ontology_graph="223p.ttl")
```

## Modeling Setup (3)

- possible to build graphs by hand, but more convenient to use the Builder
- object-oriented abstraction over the graph definition language

```python
ctx = TemplateBuilderContext(BLDG)
ctx.add_templates_from_library(nrel)
ctx.add_templates_from_library(water)
```

## Example: Simple WWTP

Consider this simple WWTP from [Villez et al 2016](http://dx.doi.org/10.1016/j.watres.2016.05.068)

Need to model
- installed equipment
- connections between them
- possible sensor positions

<img style="width: 50%" src="fig/kv1-system.png" />

## Example: Simple WWTP

```python
# create the process units
aerobic = ctx["aerobic-process-unit"](name="AEROBIC")
settler = ctx["settler"](name="SETTLER")
# connect aerobic to settler
pipe(aerobic["water-out"], settler["water-in"])
# settler sludge return goes to a junction
# make the junction
sludge_junction = ctx["junction-1to2"](name="SLUDGE_JUNCTION")
# connect the input to the junction to the sludge return of the settler with a pipe
pipe(settler["sludge-return"], sludge_junction["in1"])
```

## Example: Simple WWTP

```python
aerobic_junction = ctx["junction-2to1"](name="AEROBIC_JUNCTION")
# aerobic junction connects to input of aerobic process
pipe(aerobic_junction["out1"], aerobic["water-in"])
# pipe from settler junction to the aerobic
pipe(sludge_junction["out1"], aerobic_junction["in2"])
```

- that's it! 1 line of code for each process, pipe segment, junction

## Example: Simple WWTP

- save to a "Turtle" file for machine-to-machine transfer

```python
bldg.add_graph(ctx.compile())
graph = bldg.compile([s223.get_shape_collection()])
# this is the serialization of the model for machine-to-machine
graph.serialize("models/simple-wwtp.ttl", format="turtle")
```

Outputs:

```ttl
<urn:nrel_example/AEROBIC> a ns1:AerobicProcessUnit ;
    rdfs:label "AerobicProcessUnit" ;
    ns1:cnx <urn:nrel_example/water-in_ec975a91>,
        <urn:nrel_example/water-out_d319c7ae> ;
    ns1:connectedThrough <urn:nrel_example/name_26aefd63>,
        <urn:nrel_example/name_d6d3f618> ;
    ns1:connectedTo <urn:nrel_example/SETTLER> ;
    ns1:hasConnectionPoint <urn:nrel_example/water-in_ec975a91>,
        <urn:nrel_example/water-out_d319c7ae> .
# etc...
```

## Example: Simple WWTP

- more convenient to look at an image for human consumption

```python
# this is an automatic visualization of the model
write_to_file(graph, "models/simple-wwtp.png")
```

![](fig/kv1_mod.png)

## Application #1: Sensor Placement

- possible to compute the "process graph" for [optimal sensor placement algorithm](http://dx.doi.org/10.1016/j.watres.2016.05.068)
- in prior work, this needed to be done by hand
- now, it is a simple set of transformation rules!
  - automatically removes unnecessary detail

![](fig/Process_Graph.png)

## Application #2: Compare Design vs Measured Values

- consider example of flow rate on a tank
- vital information often spread across multiple representations:
  - Design flow rate (PFD)
  - Rated flow rate (cutsheet)
  - Measured flow rate (SCADA)
- using metadata graph, we can connect all of these together! 

## Application #2: Compare Design vs Measured Values

- attach the design and max flow rate to the tank entity definition

```turtle
:MyTank a s223:Tank ;
    s223:hasConnectionPoint :TankOutlet ;
    s223:contains :MyFlowSensor ;
    # from PFD
    ont:designFlowRate [
        ont:value 70 ;
        qudt:hasUnit unit:GAL_US-PER-MIN ;
        qudt:hasQuantityKind  qk:VolumeFlowRate ;
    ] ;
    # from cutsheet
    ont:maximumFlowRate [
        ont:value 100 ;
        qudt:hasUnit unit:GAL_US-PER-MIN ;
        qudt:hasQuantityKind  qk:VolumeFlowRate ;
    ] ;
.
```

## Application #2: Compare Design vs Measured Values

- the sensor observes the flow rate at the outlet of the tank
- here we connect to a BACnet network, but this can easily be any other SCADA solution

```turtle
:TankOutlet a s223:OutletConnectionPoint ;
    s223:hasProperty :TankFlowProperty ;
    s223:hasMedium s223:Medium-Water
.
:TankFlowProperty a s223:QuantifiableObservableProperty ;
    qudt:hasQuantityKind qk:VolumeFlowRate
.
:MyFlowSensor a s223:Sensor ;
    s223:observes :TankFlowProperty
    # from SCADA
    s223:hasExternalReference [
        a bacnet:AnalogInputObject ;
        bacnet:object-identifier "analog-input,3" ;
        ont:bacnet-property "present-value" ;
    ]
.
```

## Application #2: Compare Design vs Measured Values

- this is a SPARQL query to retrieve design, rated and measured values of tank flow rate

```sparql
SELECT ?tank ?designFlowRate ?ratedMaxFlowRate ?objectIdentifier
WHERE {
    ?tank a s223:Tank ;
          s223:designFlowRate/ont:value ?designFlowRate ;
          s223:ratedMaxFlowRate/ont:value ?ratedMaxFlowRate ;
          s223:cnx ?outlet .
    ?outlet a s223:OutletConnectionPoint ;
          s223:hasProperty ?flowProperty .
    ?sensor a s223:Sensor ;
            s223:observes ?flowProperty ;
            s223:hasExternalReference ?reference .
    ?reference a bacnet:AnalogInputObject ;
               bacnet:object-identifier ?objectIdentifier ;
               ont:bacnet-property "present-value" .
}
```

## Next Steps

- Follow development at https://github.com/DataDrivenCPS/water-ontology
- Learn more about the underlying ontology at https://open223.info